In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Cargar los datos de precios históricos de BTC
data = pd.read_csv('BitcoinHistoricalData.csv')

data

In [ ]:

# btc_data = pd.read_csv('btc_prices_2017_2020.csv')  # Cambia la ruta por la de tu archivo
# btc_test_data = pd.read_csv('btc_prices_2022_actual.csv')  # Datos de prueba desde 2022 hasta hoy

# # Preprocesamiento de los datos
# data = btc_data['Close'].values.reshape(-1, 1)  # Extraer la columna de cierre y redimensionarla
# scaler = MinMaxScaler(feature_range=(0, 1))  # Escalar los datos al rango [0, 1]
# scaled_data = scaler.fit_transform(data)  # Transformar los datos usando el escalador

# # Crear secuencias de entrenamiento
# window_size = 60  # Usaremos una ventana de 60 días para predecir el movimiento
# x_train, y_train = [], []
# for i in range(window_size, len(scaled_data)):
#     # Crear una secuencia de 60 días
#     x_train.append(scaled_data[i-window_size:i, 0])
#     # Etiquetar si el precio sube, baja o se mantiene
#     if scaled_data[i, 0] > scaled_data[i-1, 0]:
#         y_train.append([1, 0, 0])  # sube
#     elif scaled_data[i, 0] < scaled_data[i-1, 0]:
#         y_train.append([0, 1, 0])  # baja
#     else:
#         y_train.append([0, 0, 1])  # se mantiene

# # Convertir las listas a arreglos numpy para poder utilizarlas en el modelo
# x_train, y_train = np.array(x_train), np.array(y_train)
# # Reshape de x_train para que sea compatible con LSTM: (samples, timesteps, features)
# x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# # Crear el modelo LSTM
# model = Sequential()
# model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))  # Primera capa LSTM con 50 unidades
# model.add(Dropout(0.2))  # Dropout del 20% para evitar sobreajuste
# model.add(LSTM(units=50, return_sequences=False))  # Segunda capa LSTM con 50 unidades
# model.add(Dropout(0.2))  # Dropout del 20% para evitar sobreajuste
# model.add(Dense(units=25))  # Capa densa intermedia con 25 unidades
# model.add(Dense(units=3, activation='softmax'))  # Capa de salida con 3 unidades para las clases (sube, baja, se mantiene)

# # Compilar el modelo
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# # Entrenar el modelo con los datos de entrenamiento
# model.fit(x_train, y_train, epochs=10, batch_size=32)  # Entrenar por 10 épocas con un batch size de 32

# # Preprocesar los datos de prueba
# test_data = btc_test_data['Close'].values.reshape(-1, 1)  # Extraer y redimensionar la columna de cierre de los datos de prueba
# test_scaled_data = scaler.transform(test_data)  # Escalar los datos de prueba usando el mismo escalador

# # Crear secuencias para la prueba
# x_test = []
# for i in range(window_size, len(test_scaled_data)):
#     # Crear una secuencia de 60 días para cada muestra de prueba
#     x_test.append(test_scaled_data[i-window_size:i, 0])

# # Convertir la lista de secuencias de prueba a un arreglo numpy
# x_test = np.array(x_test)
# # Reshape de x_test para que sea compatible con LSTM: (samples, timesteps, features)
# x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# # Evaluar el modelo en los datos de prueba
# y_pred = model.predict(x_test)  # Predecir los movimientos del precio en los datos de prueba

# # Mostrar los resultados
# # Función para interpretar la predicción del modelo
# def interpret_prediction(prediction):
#     # Devolver la clase con la probabilidad más alta
#     if prediction[0] == max(prediction):
#         return 'Sube'
#     elif prediction[1] == max(prediction):
#         return 'Baja'
#     else:
#         return 'Se mantiene'

# # Interpretar y mostrar las predicciones
# predicted_movements = [interpret_prediction(pred) for pred in y_pred]
# print(predicted_movements)

# # Predecir el movimiento a partir del precio actual
# # Usar los últimos 60 días del conjunto de prueba para hacer una predicción
# actual_price = np.array(test_scaled_data[-window_size:]).reshape(1, window_size, 1)
# probability_vector = model.predict(actual_price)  # Predecir la probabilidad de cada clase
# print(f"Probabilidad de Sube: {probability_vector[0][0]:.2f}, Baja: {probability_vector[0][1]:.2f}, Se mantiene: {probability_vector[0][2]:.2f}")

